In [1]:
import pandas as pd
import numpy as np
import pycountry

**Generation of a Merged World Happiness Dataset (2016-2019)**

In [2]:
# Column names of the merged datasets

COUNTRY = 'Country'
YEAR = 'Year'
HAPPINESS_RANK = 'Happiness rank'
HAPPINESS_SCORE = 'Happiness score'
GDP_PER_CAPITA = 'GDP per capita'
SOCIAL_SUPPORT = 'Social support'
HEALTH = 'Life expectancy'
FREEDOM = 'Freedom'
CORRUPTION = 'Perceptions of corruption'
GENEROSITY = 'Generosity'
DYSTOPIA_RESIDUAL = 'Dystopia residual'

In [3]:
# Create a dictionary for each set of column names that can be used to make the column names consistent in all datasets
    
column_names_2015_2016 = {
    'Country': COUNTRY,
    'Year': YEAR,
    'Happiness Rank': HAPPINESS_RANK, 
    'Happiness Score': HAPPINESS_SCORE,
    'Economy (GDP per Capita)': GDP_PER_CAPITA,
    'Family': SOCIAL_SUPPORT,
    'Health (Life Expectancy)': HEALTH,
    'Freedom': FREEDOM,
    'Trust (Government Corruption)': CORRUPTION,
    'Generosity': GENEROSITY,
    'Dystopia Residual': DYSTOPIA_RESIDUAL
}
    
column_names_2017 = {
    'Country': COUNTRY,
    'Year': YEAR,
    'Happiness.Rank': HAPPINESS_RANK, 
    'Happiness.Score': HAPPINESS_SCORE,
    'Economy..GDP.per.Capita.': GDP_PER_CAPITA,
    'Family': SOCIAL_SUPPORT,
    'Health..Life.Expectancy.': HEALTH,
    'Freedom': FREEDOM,
    'Trust..Government.Corruption.': CORRUPTION,
    'Generosity': GENEROSITY,
    'Dystopia.Residual': DYSTOPIA_RESIDUAL
}
    
column_names_2018_2019 = {
    'Country or region': COUNTRY, 
    'Year': YEAR,
    'Overall rank': HAPPINESS_RANK, 
    'Score': HAPPINESS_SCORE,
    'GDP per capita': GDP_PER_CAPITA,
    'Social support': SOCIAL_SUPPORT,
    'Healthy life expectancy': HEALTH,
    'Freedom to make life choices': FREEDOM,
    'Perceptions of corruption': CORRUPTION,
    'Generosity': GENEROSITY,
}

In [4]:
world_happiness_datasets = list()

# Load World Happiness Reports for 2016-2019, and add the years to the datasets
for i in range(2015, 2020):
    df = pd.read_csv(f'./dataset/world_happiness/overall/{i}.csv')
    
    df['Year'] = i
    year_column = df.pop('Year')
    df.insert(3, 'Year', year_column)
    
    world_happiness_datasets.append(df)
    
# Process 2015 and 2016 datasets
for i in range(0, 2):
    world_happiness_datasets[i] = world_happiness_datasets[i][list(column_names_2015_2016.keys())]
    world_happiness_datasets[i] = world_happiness_datasets[i].rename(columns=column_names_2015_2016)
    
# Process 2017 dataset
world_happiness_datasets[2] = world_happiness_datasets[2][list(column_names_2017.keys())]
world_happiness_datasets[2] = world_happiness_datasets[2].rename(columns=column_names_2017)

# Process 2018 and 2019 datasets
for i in range(3, 5):
    world_happiness_datasets[i] = world_happiness_datasets[i][list(column_names_2018_2019.keys())]
    world_happiness_datasets[i] = world_happiness_datasets[i].rename(columns=column_names_2018_2019)
    
    # Calculate dystopia residual for all countries
    world_happiness_datasets[i][DYSTOPIA_RESIDUAL] = world_happiness_datasets[i][HAPPINESS_SCORE] \
    - world_happiness_datasets[i][GDP_PER_CAPITA] \
    - world_happiness_datasets[i][SOCIAL_SUPPORT] - world_happiness_datasets[i][HEALTH] \
    - world_happiness_datasets[i][FREEDOM] - world_happiness_datasets[i][CORRUPTION] \
    - world_happiness_datasets[i][GENEROSITY]

# Combine the happiness data for all years into a single dataset
merged_world_happiness_dataset = pd.concat(world_happiness_datasets, ignore_index=True)

# Sort entries by country names and years in ascending order
merged_world_happiness_dataset = merged_world_happiness_dataset.sort_values(['Country', 'Year']).drop_duplicates().reset_index(drop=True)

merged_world_happiness_dataset

,Country,Year,Happiness rank,Happiness score,GDP per capita,Social support,Life expectancy,Freedom,Perceptions of corruption,Generosity,Dystopia residual
0,Afghanistan,2015,153,3.575,0.319820,0.302850,0.303350,0.234140,0.097190,0.365100,1.952100
1,Afghanistan,2016,154,3.360,0.382270,0.110370,0.173440,0.164300,0.071120,0.312680,2.145580
2,Afghanistan,2017,141,3.794,0.401477,0.581543,0.180747,0.106180,0.061158,0.311871,2.150801
3,Afghanistan,2018,145,3.632,0.332000,0.537000,0.255000,0.085000,0.036000,0.191000,2.196000
4,Afghanistan,2019,154,3.203,0.350000,0.517000,0.361000,0.000000,0.025000,0.158000,1.792000
...,...,...,...,...,...,...,...,...,...,...,...
777,Zimbabwe,2015,115,4.610,0.271000,1.032760,0.334750,0.258610,0.080790,0.189870,2.441910
778,Zimbabwe,2016,131,4.193,0.350410,0.714780,0.159500,0.254290,0.085820,0.185030,2.442700
779,Zimbabwe,2017,138,3.875,0.375847,1.083096,0.196764,0.336384,0.095375,0.189143,1.597970
780,Zimbabwe,2018,144,3.692,0.357000,1.094000,0.248000,0.406000,0.099000,0.132000,1.356000


In [5]:
def get3countryCode(names):
    # Store the country codes that are found in a dictionary to speed up the function
    code_map = dict()
    
    res = []
    for name in names:
        try:
            try:
                code = code_map[name]
            except KeyError:
                code = pycountry.countries.search_fuzzy(name)[0].alpha_3
                code_map[name] = code
                
            res.append(code)
        except LookupError:
            res.append(None)
    
    return pd.Series(res)

In [6]:
# Add country codes
merged_world_happiness_dataset['Country code'] = get3countryCode(merged_world_happiness_dataset['Country'])

merged_world_happiness_dataset = merged_world_happiness_dataset.dropna()
merged_world_happiness_dataset

,Country,Year,Happiness rank,Happiness score,GDP per capita,Social support,Life expectancy,Freedom,Perceptions of corruption,Generosity,Dystopia residual,Country code
0,Afghanistan,2015,153,3.575,0.319820,0.302850,0.303350,0.234140,0.097190,0.365100,1.952100,AFG
1,Afghanistan,2016,154,3.360,0.382270,0.110370,0.173440,0.164300,0.071120,0.312680,2.145580,AFG
2,Afghanistan,2017,141,3.794,0.401477,0.581543,0.180747,0.106180,0.061158,0.311871,2.150801,AFG
3,Afghanistan,2018,145,3.632,0.332000,0.537000,0.255000,0.085000,0.036000,0.191000,2.196000,AFG
4,Afghanistan,2019,154,3.203,0.350000,0.517000,0.361000,0.000000,0.025000,0.158000,1.792000,AFG
...,...,...,...,...,...,...,...,...,...,...,...,...
777,Zimbabwe,2015,115,4.610,0.271000,1.032760,0.334750,0.258610,0.080790,0.189870,2.441910,ZWE
778,Zimbabwe,2016,131,4.193,0.350410,0.714780,0.159500,0.254290,0.085820,0.185030,2.442700,ZWE
779,Zimbabwe,2017,138,3.875,0.375847,1.083096,0.196764,0.336384,0.095375,0.189143,1.597970,ZWE
780,Zimbabwe,2018,144,3.692,0.357000,1.094000,0.248000,0.406000,0.099000,0.132000,1.356000,ZWE


**Removal of Developed Countries from Dataset**

In [7]:
# Developed countries as of 2018 according to the IMF
# https://www.imf.org/~/media/Files/Publications/WEO/2018/October/English/main-report/Text.ashx
developed_countries = pd.read_csv(f'./dataset/developed_countries.csv')
developed_countries['Country code'] = get3countryCode(developed_countries['Country'])

developed_countries

,Country,Country code
0,Austria,AUT
1,Greece,GRC
2,Netherlands,NLD
3,Belgium,BEL
4,Ireland,IRL
5,Portugal,PRT
6,Cyprus,CYP
7,Italy,ITA
8,Slovak Republic,SVK
9,Estonia,EST


In [8]:
developing_countries_df = merged_world_happiness_dataset[~merged_world_happiness_dataset['Country code'].isin(
    list(developed_countries['Country code']
))]

developing_countries_df

,Country,Year,Happiness rank,Happiness score,GDP per capita,Social support,Life expectancy,Freedom,Perceptions of corruption,Generosity,Dystopia residual,Country code
0,Afghanistan,2015,153,3.575,0.319820,0.302850,0.303350,0.234140,0.097190,0.365100,1.952100,AFG
1,Afghanistan,2016,154,3.360,0.382270,0.110370,0.173440,0.164300,0.071120,0.312680,2.145580,AFG
2,Afghanistan,2017,141,3.794,0.401477,0.581543,0.180747,0.106180,0.061158,0.311871,2.150801,AFG
3,Afghanistan,2018,145,3.632,0.332000,0.537000,0.255000,0.085000,0.036000,0.191000,2.196000,AFG
4,Afghanistan,2019,154,3.203,0.350000,0.517000,0.361000,0.000000,0.025000,0.158000,1.792000,AFG
...,...,...,...,...,...,...,...,...,...,...,...,...
777,Zimbabwe,2015,115,4.610,0.271000,1.032760,0.334750,0.258610,0.080790,0.189870,2.441910,ZWE
778,Zimbabwe,2016,131,4.193,0.350410,0.714780,0.159500,0.254290,0.085820,0.185030,2.442700,ZWE
779,Zimbabwe,2017,138,3.875,0.375847,1.083096,0.196764,0.336384,0.095375,0.189143,1.597970,ZWE
780,Zimbabwe,2018,144,3.692,0.357000,1.094000,0.248000,0.406000,0.099000,0.132000,1.356000,ZWE


In [9]:
developed_countries_df = merged_world_happiness_dataset[merged_world_happiness_dataset['Country code'].isin(
    list(developed_countries['Country code']
))]

developed_countries_df

,Country,Year,Happiness rank,Happiness score,GDP per capita,Social support,Life expectancy,Freedom,Perceptions of corruption,Generosity,Dystopia residual,Country code
29,Australia,2015,10,7.284,1.333580,1.309230,0.931560,0.651240,0.356370,0.435620,2.266460,AUS
30,Australia,2016,9,7.313,1.444430,1.104760,0.851200,0.568370,0.323310,0.474070,2.546500,AUS
31,Australia,2017,10,7.284,1.484415,1.510042,0.843887,0.601607,0.301184,0.477699,2.065211,AUS
32,Australia,2018,10,7.272,1.340000,1.573000,0.910000,0.647000,0.302000,0.361000,2.139000,AUS
33,Australia,2019,11,7.228,1.372000,1.548000,1.036000,0.557000,0.290000,0.332000,2.093000,AUS
...,...,...,...,...,...,...,...,...,...,...,...,...
742,United States,2015,15,7.119,1.394510,1.247110,0.861790,0.546040,0.158900,0.401050,2.510110,USA
743,United States,2016,13,7.104,1.507960,1.047820,0.779000,0.481630,0.148680,0.410770,2.727820,USA
744,United States,2017,14,6.993,1.546259,1.419921,0.774287,0.505741,0.135639,0.392579,2.218113,USA
745,United States,2018,18,6.886,1.398000,1.471000,0.819000,0.547000,0.133000,0.291000,2.227000,USA


**Generation of a Simplified SDG Dataset with Indicators of Urbanization**

In [10]:
# Column names of the dataset

SDG_COLUMNS = [
    'Country Code',
    'Indicator Name',
    '2014',
    '2015',
    '2016',
    '2017',
    '2018',
]

In [11]:
# Load SDG dataset and extract the relevant columns from it
df = pd.read_csv(f'./dataset/world_happiness/sustainable_develop_goals/sdg-csv-zip-7-mb-/SDGData.csv')
df = df[SDG_COLUMNS]

df

,Country Code,Indicator Name,2014,2015,2016,2017,2018
0,ARB,Access to clean fuels and technologies for coo...,83.954293,84.230630,84.570425,NaN,NaN
1,ARB,Access to electricity (% of population),88.076774,88.517967,88.768654,NaN,NaN
2,ARB,"Access to electricity, rural (% of rural popul...",77.487377,78.564439,78.958780,NaN,NaN
3,ARB,"Access to electricity, urban (% of urban popul...",96.411432,96.574021,96.704393,NaN,NaN
4,ARB,Account ownership at a financial institution o...,30.277130,NaN,NaN,37.165211,NaN
...,...,...,...,...,...,...,...
98620,ZWE,"Wage and salaried workers, total (% of total e...",33.979000,33.898998,33.848999,33.888000,33.872002
98621,ZWE,"Water productivity, total (constant 2010 US$ G...",NaN,NaN,NaN,NaN,NaN
98622,ZWE,Women making their own informed decisions rega...,NaN,59.900000,NaN,NaN,NaN
98623,ZWE,Women who were first married by age 15 (% of w...,NaN,3.700000,NaN,NaN,NaN


In [12]:
# List of unique development indicators
set(df['Indicator Name'].to_list())

{'Access to clean fuels and technologies for cooking (% of population)',
 'Access to electricity (% of population)',
 'Access to electricity, rural (% of rural population)',
 'Access to electricity, urban (% of urban population)',
 'Account ownership at a financial institution or with a mobile-money-service provider (% of population ages 15+)',
 'Account ownership at a financial institution or with a mobile-money-service provider, female (% of population ages 15+)',
 'Account ownership at a financial institution or with a mobile-money-service provider, male (% of population ages 15+)',
 'Account ownership at a financial institution or with a mobile-money-service provider, older adults (% of population ages 25+)',
 'Account ownership at a financial institution or with a mobile-money-service provider, poorest 40% (% of population ages 15+)',
 'Account ownership at a financial institution or with a mobile-money-service provider, primary education or less (% of population ages 15+)',
 'Acc

In [13]:
# Indicators which are expected to indicate urbanization if their values are high
URBANIZATION_INDICATORS = [
    'Air transport, freight (million ton-km)',
    'Air transport, passengers carried',
    'Annual freshwater withdrawals, industry (% of total freshwater withdrawal)',
    'CO2 emissions (metric tons per capita)',
    'Commercial bank branches (per 100,000 adults)',
    'Employment in industry (% of total employment) (modeled ILO estimate)',
    'Employment in services (% of total employment) (modeled ILO estimate)',
    'Industry (including construction), value added per worker (constant 2010 US$)',
    'Manufacturing, value added (% of GDP)',
    'Medium and high-tech industry (% manufacturing value added)',
    'Railways, goods transported (million ton-km)',
    'Railways, passengers carried (million passenger-km)',
    'Urban population (% of total)',
    
    # Other indicators that could be used (internet access tends to be better in densely populated areas, more universities in
    # cities)
    'Individuals using the Internet (% of population)',
    "Educational attainment, competed at least Bachelor's or equivalent, population 25+, total (%) (cumulative)",
    "Educational attainment, competed at least Master's or equivalent, population 25+, total (%) (cumulative)",
]

In [14]:
# Indiccators which are expected to indicate urbanization if their values are low
ANTI_URBANIZATION_INDICATORS = [
    'Agriculture, forestry, and fishing, value added per worker (constant 2010 US$)',
    'Annual freshwater withdrawals, agriculture (% of total freshwater withdrawal)',
    'Employment in agriculture (% of total employment) (modeled ILO estimate)',
    'Forest area (% of land area)',
]

In [15]:
# Find rows containing urbanization and anti-urbanization indicators
sdg_dataset = df[df['Indicator Name'].isin(URBANIZATION_INDICATORS + ANTI_URBANIZATION_INDICATORS)]

sdg_dataset

,Country Code,Indicator Name,2014,2015,2016,2017,2018
18,ARB,"Agriculture, forestry, and fishing, value adde...",5.547096e+03,5.627784e+03,5.677982e+03,NaN,NaN
19,ARB,"Air transport, freight (million ton-km)",2.469890e+04,2.591846e+04,2.786970e+04,3.024622e+04,NaN
20,ARB,"Air transport, passengers carried",1.816273e+08,1.959570e+08,2.143187e+08,2.240003e+08,NaN
21,ARB,"Annual freshwater withdrawals, agriculture (% ...",NaN,NaN,NaN,NaN,NaN
23,ARB,"Annual freshwater withdrawals, industry (% of ...",NaN,3.442000e+00,NaN,NaN,NaN
...,...,...,...,...,...,...,...
98433,ZWE,"Manufacturing, value added (% of GDP)",1.259076e+01,1.188860e+01,1.159602e+01,1.101701e+01,NaN
98437,ZWE,Medium and high-tech industry (% manufacturing...,NaN,NaN,NaN,NaN,NaN
98542,ZWE,"Railways, goods transported (million ton-km)",NaN,NaN,NaN,NaN,NaN
98543,ZWE,"Railways, passengers carried (million passenge...",NaN,NaN,NaN,NaN,NaN


In [16]:
# Tidy the dataset for sustainable development goals

sdg_dataset = pd.wide_to_long(
    sdg_dataset,
    stubnames=[''],
    sep='',
    i=['Country Code', 'Indicator Name'],
    j='Year'
).dropna().reset_index().rename(columns={'': 'Indicator Value'})

sdg_dataset

,Country Code,Indicator Name,Year,Indicator Value
0,ARB,"Agriculture, forestry, and fishing, value adde...",2014,5547.095506
1,ARB,"Agriculture, forestry, and fishing, value adde...",2015,5627.784460
2,ARB,"Agriculture, forestry, and fishing, value adde...",2016,5677.981646
3,ARB,"Air transport, freight (million ton-km)",2014,24698.899692
4,ARB,"Air transport, freight (million ton-km)",2015,25918.457903
...,...,...,...,...
12557,ZWE,"Manufacturing, value added (% of GDP)",2017,11.017009
12558,ZWE,Urban population (% of total),2014,32.504000
12559,ZWE,Urban population (% of total),2015,32.385000
12560,ZWE,Urban population (% of total),2016,32.296000


In [17]:
tidied_sdg_df = pd.pivot_table(
    sdg_dataset, 
    index=['Country Code', 'Year'], 
    columns=['Indicator Name'], 
    values='Indicator Value'
).reset_index().rename(columns={'Country Code': 'Country code'})

tidied_sdg_df

Indicator Name,Country code,Year,"Agriculture, forestry, and fishing, value added per worker (constant 2010 US$)","Air transport, freight (million ton-km)","Air transport, passengers carried","Annual freshwater withdrawals, agriculture (% of total freshwater withdrawal)","Annual freshwater withdrawals, industry (% of total freshwater withdrawal)",CO2 emissions (metric tons per capita),"Commercial bank branches (per 100,000 adults)","Educational attainment, competed at least Bachelor's or equivalent, population 25+, total (%) (cumulative)",...,Employment in industry (% of total employment) (modeled ILO estimate),Employment in services (% of total employment) (modeled ILO estimate),Forest area (% of land area),Individuals using the Internet (% of population),"Industry (including construction), value added per worker (constant 2010 US$)","Manufacturing, value added (% of GDP)",Medium and high-tech industry (% manufacturing value added),"Railways, goods transported (million ton-km)","Railways, passengers carried (million passenger-km)",Urban population (% of total)
0,ABW,2014,NaN,NaN,NaN,NaN,NaN,8.408363,19.061688,NaN,...,NaN,NaN,2.333333,83.780000,NaN,NaN,NaN,NaN,NaN,43.041
1,ABW,2015,NaN,NaN,NaN,NaN,NaN,NaN,18.862141,NaN,...,NaN,NaN,2.333333,88.661227,NaN,NaN,NaN,NaN,NaN,43.108
2,ABW,2016,NaN,NaN,NaN,NaN,NaN,NaN,19.836408,NaN,...,NaN,NaN,2.333333,93.542454,NaN,NaN,NaN,NaN,NaN,43.192
3,ABW,2017,NaN,NaN,NaN,NaN,NaN,NaN,18.516375,NaN,...,NaN,NaN,NaN,97.170000,NaN,NaN,NaN,NaN,NaN,43.293
4,AFG,2014,1028.653404,34.283472,2209428.0,NaN,NaN,0.299445,2.332229,NaN,...,17.455999,43.897999,2.067825,7.000000,2123.921733,11.477588,9.507434,NaN,NaN,24.587
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1278,ZWE,2014,318.569374,0.710490,301260.0,NaN,NaN,0.779956,12.811761,NaN,...,7.352000,25.398001,37.157556,16.364740,6066.112260,12.590765,NaN,NaN,NaN,32.504
1279,ZWE,2015,294.738272,0.796107,370165.0,NaN,NaN,NaN,5.165438,NaN,...,7.290000,25.589001,36.350006,22.742818,5962.002633,11.888599,NaN,NaN,NaN,32.385
1280,ZWE,2016,275.073784,0.804947,378803.0,NaN,NaN,NaN,5.260135,NaN,...,7.218000,25.590000,35.542457,23.119989,5930.642813,11.596020,NaN,NaN,NaN,32.296
1281,ZWE,2017,294.061198,0.665035,282539.0,NaN,NaN,NaN,4.424425,NaN,...,7.254000,25.673000,NaN,27.055488,5866.001706,11.017009,NaN,NaN,NaN,32.237


**Merging the World Happiness Report and Sustainable Development Goals Datasets**

In [18]:
# Developing countries
developing_countries_df = developing_countries_df.merge(tidied_sdg_df, on=['Country code', 'Year'])
country_codes = developing_countries_df.pop('Country code')
developing_countries_df.insert(1, 'Country code', country_codes)

developing_countries_df

,Country,Country code,Year,Happiness rank,Happiness score,GDP per capita,Social support,Life expectancy,Freedom,Perceptions of corruption,...,Employment in industry (% of total employment) (modeled ILO estimate),Employment in services (% of total employment) (modeled ILO estimate),Forest area (% of land area),Individuals using the Internet (% of population),"Industry (including construction), value added per worker (constant 2010 US$)","Manufacturing, value added (% of GDP)",Medium and high-tech industry (% manufacturing value added),"Railways, goods transported (million ton-km)","Railways, passengers carried (million passenger-km)",Urban population (% of total)
0,Afghanistan,AFG,2015,153,3.575,0.319820,0.302850,0.303350,0.234140,0.097190,...,17.646999,43.674999,2.067825,8.260000,2079.555916,11.420006,9.507434,NaN,NaN,24.803
1,Afghanistan,AFG,2016,154,3.360,0.382270,0.110370,0.173440,0.164300,0.071120,...,17.424999,43.756001,2.067825,10.595726,1999.566153,11.370465,NaN,NaN,NaN,25.020
2,Afghanistan,AFG,2017,141,3.794,0.401477,0.581543,0.180747,0.106180,0.061158,...,17.457001,43.768002,NaN,11.447688,1923.393447,11.102526,NaN,NaN,NaN,25.250
3,Afghanistan,AFG,2018,145,3.632,0.332000,0.537000,0.255000,0.085000,0.036000,...,17.596001,43.816002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Albania,ALB,2015,95,4.959,0.878670,0.804340,0.813250,0.357330,0.064130,...,18.617001,40.020000,28.156934,63.252933,13648.254549,5.671519,6.690696,NaN,NaN,57.434
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
452,Zambia,ZMB,2018,125,4.377,0.562000,1.047000,0.295000,0.503000,0.082000,...,10.683000,35.401001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
453,Zimbabwe,ZWE,2015,115,4.610,0.271000,1.032760,0.334750,0.258610,0.080790,...,7.290000,25.589001,36.350006,22.742818,5962.002633,11.888599,NaN,NaN,NaN,32.385
454,Zimbabwe,ZWE,2016,131,4.193,0.350410,0.714780,0.159500,0.254290,0.085820,...,7.218000,25.590000,35.542457,23.119989,5930.642813,11.596020,NaN,NaN,NaN,32.296
455,Zimbabwe,ZWE,2017,138,3.875,0.375847,1.083096,0.196764,0.336384,0.095375,...,7.254000,25.673000,NaN,27.055488,5866.001706,11.017009,NaN,NaN,NaN,32.237


In [ ]:
# Developed countries
developed_countries_df = developed_countries_df.merge(tidied_sdg_df, on=['Country code', 'Year'])
country_codes = developed_countries_df.pop('Country code')
developing_countries_df.insert(1, 'Country code', country_codes)

developing_countries_df